# Assignment #1 Solution

## Step 1: Load and select the Philadelphia data

In [3]:
import pandas as pd

In [55]:
path = "./data/Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv"
data = pd.read_csv(path)

In [56]:
data.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,193814.0,...,1036773.0,1039292.0,1051475.0,1052389.0,1062227.0,1073321.0,1088262.0,1101458.0,1110816.0,1117997.0
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,262402.0,...,497434.0,499976.0,502451.0,504692.0,507078.0,508667.0,510851.0,513170.0,516360.0,519384.0
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,261664.0,...,1105905.0,1101323.0,1092186.0,1083389.0,1083178.0,1081122.0,1084203.0,1086301.0,1088716.0,1088755.0
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,201649.0,...,345982.0,349035.0,353498.0,358004.0,363402.0,369313.0,376254.0,383839.0,392947.0,403712.0
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,353678.0,...,635561.0,638519.0,641360.0,643819.0,646973.0,649403.0,652545.0,654806.0,658509.0,661889.0


To select Philadelphia data, we have to select by the `City` column and the `State` column.

In [57]:
selection = (data['City'] == 'Philadelphia') & (data['State'] == 'PA')
data = data.loc[selection]

In [58]:
data.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31
125,65810,126,19143,Zip,PA,PA,Philadelphia,Philadelphia-Camden-Wilmington,Philadelphia County,53524.0,...,157194.0,160135.0,163269.0,165590.0,167517.0,169735.0,171993.0,174541.0,176363.0,177678.0
247,65779,249,19111,Zip,PA,PA,Philadelphia,Philadelphia-Camden-Wilmington,Philadelphia County,84371.0,...,224458.0,227703.0,231011.0,234006.0,236974.0,239798.0,242877.0,246638.0,250739.0,254296.0
338,65791,340,19124,Zip,PA,PA,Philadelphia,Philadelphia-Camden-Wilmington,Philadelphia County,44683.0,...,133674.0,135605.0,137586.0,139327.0,141039.0,142843.0,144890.0,147221.0,149610.0,151462.0
423,65787,426,19120,Zip,PA,PA,Philadelphia,Philadelphia-Camden-Wilmington,Philadelphia County,56842.0,...,135938.0,137791.0,139907.0,141986.0,144248.0,146505.0,148650.0,151139.0,153847.0,156424.0
509,65772,512,19104,Zip,PA,PA,Philadelphia,Philadelphia-Camden-Wilmington,Philadelphia County,83638.0,...,200585.0,205137.0,207906.0,210565.0,211314.0,213983.0,216661.0,220064.0,221822.0,221487.0


## Step 2: Calculate the annual ZHVI per ZIP code

First, remove unnecessary columns from the data frame.

In [59]:
data = data.drop(
    [
        "RegionID",
        "RegionType",
        "City",
        "StateName",
        "State",
        "Metro",
        "CountyName",
        "SizeRank",
    ],
    axis=1,
)

Next, melt the data frame into long format.

In [60]:
melted = data.melt(id_vars=["RegionName"], value_name='ZHVI', var_name='Date')

In [61]:
melted

,RegionName,Date,ZHVI
0,19143,1996-01-31,53524.0
1,19111,1996-01-31,84371.0
2,19124,1996-01-31,44683.0
3,19120,1996-01-31,56842.0
4,19104,1996-01-31,83638.0
...,...,...,...
14117,19153,2021-07-31,216462.0
14118,19118,2021-07-31,685303.0
14119,19102,2021-07-31,355060.0
14120,19127,2021-07-31,308595.0


Convert the `Date` from a string to a pandas `Datetime` object, and then extract the year from the data and store it as the `Year` column.

In [62]:
melted['Date'] = pd.to_datetime(melted['Date'])
melted['Year'] = melted['Date'].dt.year

To calculate the annual ZHVI: 

- group by ZIP code (`RegionName`) and year (`Year`)
- take the average of the `ZVHI` column for each group

In [63]:
annual_ZHVI = melted.groupby(['RegionName', 'Year'])['ZHVI'].mean()

In [64]:
annual_ZHVI.head()

RegionName  Year
19102       1996    102083.666667
            1997    105159.583333
            1998    109694.166667
            1999    130578.083333
            2000    155851.916667
Name: ZHVI, dtype: float64

Convert from a Series object to a DataFrame by resetting the index.

In [65]:
annual_ZHVI = annual_ZHVI.reset_index()

In [66]:
annual_ZHVI.head()

,RegionName,Year,ZHVI
0,19102,1996,102083.666667
1,19102,1997,105159.583333
2,19102,1998,109694.166667
3,19102,1999,130578.083333
4,19102,2000,155851.916667


## Step 3: Find the ZIP codes with max ZHVI for each year

We need to calculate a quantity for each year, so the first step is to group the data by the `Year` column.

In [67]:
year_grps = annual_ZHVI.groupby('Year')
print(year_grps)

Next, we'll define a function that calculates the ZIP code with the max ZHVI for each year's data.

In [68]:
def get_max_ZHVI_zip(grp):
    
    # this will give you the index of the row that has
    # the max ZHVI value
    idx_max = grp['ZHVI'].idxmax()
    
    # return the RegionName in the row with the max ZVHI
    return grp.loc[idx_max, ["RegionName", "ZHVI"]]

In [69]:
max_ZIPs = year_grps.apply(get_max_ZHVI_zip)

In [70]:
max_ZIPs

,RegionName,ZHVI
Year,,
1996,19118.0,208438.000000
1997,19118.0,215315.750000
1998,19118.0,212398.250000
1999,19118.0,223437.000000
2000,19118.0,253985.750000
2001,19118.0,283068.916667
2002,19118.0,314700.000000
2003,19118.0,361547.000000
2004,19118.0,412069.666667
